# Hola, mundo en LangChain

## Instalar librerías principales y configuración de API Key de OpenAI

In [2]:
%%capture
!pip install -U langchain langchain-openai langchain-community langchain pypdf openai chromadb tiktoken langchain-community langchain-text-splitters langchain.vectorstores langchain.chains langchain.chat_models

In [3]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")


## Carga de documents

In [5]:
import requests
from langchain_community.document_loaders import PyPDFLoader

urls = [
    'https://arxiv.org/pdf/2306.06031v1.pdf',
    'https://arxiv.org/pdf/2306.12156v1.pdf',
    'https://arxiv.org/pdf/2306.14289v1.pdf',
    'https://arxiv.org/pdf/2305.10973v1.pdf',
    'https://arxiv.org/pdf/2306.13643v1.pdf'
]

ml_papers = []

for i, url in enumerate(urls):
    response = requests.get(url)
    filename = f'paper{i+1}.pdf'
    with open(filename, 'wb') as f:
        f.write(response.content)
        print(f'Descargado {filename}')

        loader = PyPDFLoader(filename)
        data = loader.load()
        ml_papers.extend(data)

# Utiliza la lista ml_papers para acceder a los elementos de todos los documentos descargados
print('Contenido de ml_papers:')
print()

Descargado paper1.pdf
Descargado paper2.pdf
Descargado paper3.pdf
Descargado paper4.pdf
Descargado paper5.pdf
Contenido de ml_papers:



In [6]:
type(ml_papers), len(ml_papers), ml_papers[3]

(list,
 57,
 Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-12T00:32:18+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-12T00:32:18+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'templateversion': 'IJCAI.2023.0', 'title': '', 'trapped': '/False', 'source': 'paper1.pdf', 'total_pages': 7, 'page': 3, 'page_label': '4'}, page_content='Figure 1: FinGPT Framework.\n4.1 Data Sources\nThe first stage of the FinGPT pipeline involves the collec-\ntion of extensive financial data from a wide array of online\nsources. These include, but are not limited to:\n• Financial news: Websites such as Reuters, CNBC, Yahoo\nFinance, among others, are rich sources of financial news\nand market updates. These sites provide valuable informa-\ntion on market trends, company earnings, macroeconomic\nindicators, and other financial events.\n• Social media: Pl

## Split de documents

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len
    )

documents = text_splitter.split_documents(ml_papers)

In [8]:
len(documents), documents[10]

(211,
 Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-06-12T00:32:18+00:00', 'author': '', 'keywords': '', 'moddate': '2023-06-12T00:32:18+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'templateversion': 'IJCAI.2023.0', 'title': '', 'trapped': '/False', 'source': 'paper1.pdf', 'total_pages': 7, 'page': 2, 'page_label': '3'}, page_content='highly volatile, changing rapidly in response to news events\nor market movements.\nTrends, often observable through websites like Seeking\nAlpha, Google Trends, and other finance-oriented blogs and\nforums, offer critical insights into market movements and in-\nvestment strategies. They feature:\n• Analyst perspectives: These platforms provide access to\nmarket predictions and investment advice from seasoned\nfinancial analysts and experts.\n• Market sentiment: The discourse on these platforms can\nreflect th

## Embeddings e ingesta a base de datos vectorial

In [9]:

from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002")

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
    )

C:\Users\angel\AppData\Local\Temp\ipykernel_12916\2730905488.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(


## Modelos de chat y cadenas para consulta de información

In [10]:
from langchain_openai import ChatOpenAI
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    model="gpt-3.5-turbo",
    temperature=0
)

prompt = ChatPromptTemplate.from_template(
    """Usa el contexto para responder la pregunta.

    Contexto:
    {context}

    Pregunta:
    {input}
    """
)

document_chain = create_stuff_documents_chain(llm, prompt)

qa_chain = create_retrieval_chain(
    retriever,
    document_chain
)


In [13]:
# response = qa_chain.invoke({"input": "¿Qué es LangChain?"})
# print(response["answer"])

query = "qué es fingpt?"
qa_chain.invoke({"input": query})["answer"]


'FinGPT es un modelo de lenguaje financiero de código abierto que utiliza un enfoque centrado en los datos y un marco de extremo a extremo para abordar los desafíos en el procesamiento del lenguaje natural en el ámbito financiero. Su objetivo es estimular la innovación, democratizar los modelos de lenguaje financiero y desbloquear nuevas oportunidades en finanzas abiertas.'

In [14]:
query = "qué hace complicado entrenar un modelo como el fingpt?"
qa_chain.invoke({"input": query})["answer"]

'Entrenar un modelo como FinGPT es complicado debido a su intensivo requerimiento computacional y al alto costo asociado. Por ejemplo, el modelo BloombergGPT utilizó aproximadamente 1.3 millones de horas de GPU para su entrenamiento, lo que se traduce en un costo de alrededor de $3 millones. En contraste, FinGPT presenta una solución más accesible al enfocarse en la adaptación ligera de LLMs de código abierto, lo que reduce significativamente los costos de adaptación y los requisitos computacionales.'

In [15]:
query = "qué es fast segment?"
qa_chain.invoke({"input": query})["answer"]

'Fast Segment es un modelo ligero de segmentación de objetos que busca hacer que SAM sea compatible con dispositivos móviles al reemplazar el codificador de imágenes pesado con uno más ligero. Utiliza un enfoque de destilación desacoplada para mejorar su rendimiento cuando hay fuentes de entrenamiento limitadas.'

In [16]:
query = "cuál es la diferencia entre fast sam y mobile sam?"
qa_chain.invoke({"input": query})["answer"]

'La diferencia principal entre FastSAM y MobileSAM es que MobileSAM es más rápido y más pequeño en tamaño en comparación con FastSAM. MobileSAM tiene 10M parámetros, mientras que FastSAM tiene 68M parámetros. Además, MobileSAM procesa una imagen en 10ms, mientras que FastSAM tarda 40ms.'